In [1]:
%cd ~/Code/bioplnn

[Errno 2] No such file or directory: '/home/valmiki/Code/bioplnn'
/rdma/vast-rdma/user/valmiki/bioplnn/examples


/om2/user/valmiki/miniconda/envs/pytorch/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [2]:
%cd /om2/user/valmiki/bioplnn/

/rdma/vast-rdma/user/valmiki/bioplnn


/om2/user/valmiki/miniconda/envs/pytorch/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
import json
import os

import hydra
import numpy as np
import torch
from omegaconf import DictConfig, OmegaConf
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

from bioplnn.datasets.ei import qCLEVRDataset
from bioplnn.models.ei import Conv2dEIRNN
from bioplnn.utils import AttrDict, rescale
import pandas as pd
from PIL import Image
from IPython.display import display
import yaml
from collections import defaultdict
import pickle
from bioplnn.models.simple_attn_model import (
    BaselineConvRNN,
    attnCNNImplicit,
    resnetImplicit,
    txferImplicit,
)

In [4]:
config = yaml.safe_load(open("config/config_ei_test.yaml"))
config = AttrDict(config)
num_samples = 20
visualize = False

In [ ]:
for model_class in [
    # "baseline",
    # "attnCNNImplicit",
    # "resnetImplicit",
    "txferImplicitVIT",
    # "txferImplicitSwinT",
]:

In [12]:
model = Conv2dEIRNN(**config.model)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

1889962

In [5]:
clevr_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Lambda(rescale),
        transforms.Resize(config.model.input_size),
    ]
)
val_dataset = qCLEVRDataset(
    data_root=config.data.root,
    assets_path=config.data.assets_path,
    clevr_transforms=clevr_transforms,
    return_images=config.save_results,
    split="valid",
    holdout=config.data.holdout,
    mode=config.data.mode,
    primitive=config.data.primitive,
    num_workers=config.data.num_workers,
)

df = pd.read_json("outputs/predictions/whole-frost-113_in_dist.json")

*** Holding out: []
*** Mode: every


In [7]:
df.head()

,preds,labels,accs,image_paths,modes,cues
0,4,4,1,data/qclevr/valid_color/images/qCLEVR_color_tr...,color,cyan
1,5,5,1,data/qclevr/valid_color/images/qCLEVR_color_tr...,color,red
2,0,0,1,data/qclevr/valid_color/images/qCLEVR_color_tr...,color,cyan
3,2,2,1,data/qclevr/valid_color/images/qCLEVR_color_tr...,color,cyan
4,3,3,1,data/qclevr/valid_color/images/qCLEVR_color_tr...,color,red


In [ ]:
results = {}
for mode in ["color", "shape", "conjunction"]:
    results[mode] = {}
    for acc in [0, 1]:
        results[mode][acc] = []
        print(f"Mode: {mode}, Acc: {acc == 1}")
        df_mode = df[(df["modes"] == mode) & (df["accs"] == acc)]
        for i in np.random.randint(0, len(df_mode), num_samples):
            img = Image.open(df_mode["image_paths"].iloc[i])
            img = img.convert("RGB")
            cue_str = df_mode["cues"].iloc[i]
            if mode == "color":
                cue = img.copy()
                cue.paste(val_dataset.color_dict[cue_str], [0, 0, cue.size[0], cue.size[1]])
            elif mode == "shape":
                cue = val_dataset.gen_shape(img, cue_str)
            elif mode == "conjunction":
                cue_str = cue_str.split("_")
                cue = val_dataset.gen_conjunction_trial(img, cue_str)
            else:
                raise NotImplementedError
            result = {
                "img": img,
                "cue": cue,
                "pred": df_mode.iloc[i],
            }
            results[mode][acc].append(result)

            if visualize:
                print(df_mode.iloc[i])
                display(img)
                display(cue)
            
        print("-"*50)

In [ ]:
pickle.dump(results, open("outputs/predictions/whole-frost-113_imgs.pickle", "wb"))

In [ ]:
df_mode.iloc[0].to_dict()

In [4]:
pickle.load(open("outputs/predictions/whole-frost-113_imgs.pickle", "rb"))

{'color': {0: [{'img': <PIL.Image.Image image mode=RGB size=320x240>,
    'cue': <PIL.Image.Image image mode=RGB size=320x240>,
    'pred': preds                                                          3
    labels                                                         4
    accs                                                           0
    image_paths    data/qclevr/valid_color/images/qCLEVR_color_tr...
    modes                                                      color
    cues                                                         red
    Name: 3151, dtype: object},
   {'img': <PIL.Image.Image image mode=RGB size=320x240>,
    'cue': <PIL.Image.Image image mode=RGB size=320x240>,
    'pred': preds                                                          4
    labels                                                         5
    accs                                                           0
    image_paths    data/qclevr/valid_color/images/qCLEVR_color_tr...
    modes         

In [ ]:
import json
from glob import glob

with open(glob("outputs/predictions/*.json")[0]) as f:
    results = json.load(f)

In [ ]:
from bioplnn.datasets.ei import get_qclevr_dataloaders

train_loader, val_loader = get_qclevr_dataloaders(
        data_root="data/qclevr",
        assets_path="data/CLEVR_v1.0/cues",
        train_batch_size=256,
        val_batch_size=64,
        resolution=(128, 128),
        holdout=["green", "blue"],
        mode="color",
        primitive=True,
        num_workers=0,
        seed=None,
    )

In [ ]:
sample = next(iter(train_loader))

In [ ]:
sample[2].min()

In [ ]:
from torch import nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR

model = nn.Sequential(
    nn.Linear(1, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
)

optimizer = AdamW(model.parameters(), lr=1)


In [ ]:
optimizer.state_dict()

In [ ]:
scheduler.super

In [ ]:
scheduler = OneCycleLR(optimizer, max_lr=1e-2, total_steps=1000)

In [ ]:
for i in range(1000):
    optimizer.step()
    scheduler.step()

In [ ]:
import os

os.makedirs("checkpoints/ei", exist_ok=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import glob
import scipy
import os
from bioplnn.topography import TopographicalRNN
from bioplnn.dataset import CIFAR10_V1
import matplotlib.pyplot as plt
from bioplnn.utils import flatten_indices
import torch_sparse
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def build_graph(path):
    connection = {}
    coordinates = []
    Nunits = []
    Adj = []
    weight_path = glob.glob(os.path.join(path, "*.npz"))[0]
    sparse_connection = scipy.sparse.load_npz(weight_path)
    mask_path = glob.glob(os.path.join(path, "*.npy"))
    if len(mask_path) == 1:
        masks = np.load(mask_path[0])
        layers = [np.stack(mask.nonzero()) for mask in masks[:-1]]
        Ny = masks.shape[2]
    else:
        layers = ["V1_indices", "V2_indices", "V3_indices", "V4_indices"]
        layers = [
            np.load(os.path.join(path, f"{layer}.npy")) for layer in layers
        ]
        Ny = 300

    for coor in layers:
        coordinates.append(coor[0] * Ny + coor[1])

    for i, incoming in enumerate(coordinates[:-1]):
        adj = torch.from_numpy(
            sparse_connection[coordinates[i + 1]][:, incoming].toarray()
        ).float()
        Adj.append(adj)

    return Adj

In [ ]:
path = "connection"
sparse_connection = (
    torch.load(os.path.join(path, "sparse.pt")).coalesce().to(device)
)

In [ ]:
indices = sparse_connection.indices().to(device)
identity = indices.unique().tile(2, 1)
indices = torch.cat([indices, identity], 1)
values = torch.ones(indices.shape[1], device=device)

In [ ]:
w = torch.sparse_coo_tensor(
    indices, values, sparse_connection.shape, device=device
).coalesce()

In [ ]:
inverse_indices = torch.stack((indices[1], indices[0]))
w_inv = torch.sparse_coo_tensor(
    inverse_indices, values, sparse_connection.shape, device=device
).coalesce()

In [ ]:
indices[0].unique().shape

In [ ]:
sparse_connection.values()

In [ ]:
connections = sparse_connection.count_nonzero(dim=1)
connections[connections.nonzero(as_tuple=True)].float().mean()

In [ ]:
layers = ["V1_indices", "V2_indices", "V3_indices", "V4_indices"]
coordinates = [
    torch.tensor(np.load(os.path.join(path, f"{layer}.npy")))
    for layer in layers
]
Ny = 300
coordinates1d = [flatten_indices(coor) for coor in coordinates]
for coor, layer in zip(coordinates, layers):
    torch.save(coor, os.path.join(path, f"{layer}.pt"))
for coor, layer in zip(coordinates1d, layers):
    torch.save(coor, os.path.join(path, f"{layer}_flat.pt"))

In [ ]:
x = torch.zeros(16, 3, 1024)
x = x.flatten(2)
print(x.shape)
x.mean(dim=1).shape

In [ ]:
for layer in layers:
    os.remove(os.path.join(path, f"{layer}_flat.pt"))

In [ ]:
sparse_connection = sparse_connection.to("cuda")

In [ ]:
indices = sparse_connection.indices()
indices[1, :].unique()

In [ ]:
weight_path = os.path.join(path, "sparse.npz")
sparse_connection = scipy.sparse.load_npz(weight_path)

In [ ]:
values = torch.ones_like(coordinates[0]).float()

In [ ]:
pic = torch.ones(150, 300)
for i, coordinate in enumerate(coordinates):
    pic[coordinate[0], coordinate[1]] = 1 / (i + 2)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(pic)

In [ ]:
torch.sparse_coo_tensor(
    coordinates[0], values, (150, 300), device="cuda"
).to_dense()